In [1]:
from gensim.corpora.wikicorpus import WikiCorpus
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from pprint import pprint
import multiprocessing
import gensim
print(gensim.__version__) # 0.12.4

import pandas as pd
from ast import literal_eval

0.12.4


## My own data

In [2]:
df = pd.read_csv('processed_1.csv')
df.sample(5)

,r_id,doc_id,specialty,gender,S,P,H,K,review
12987,2777.3,2777,brain,m,4,5,5,5,"['seeing', '10', 'year', 'he', 'absolute', 'to..."
19718,4156.1,4156,-1,m,5,5,5,5,"['tr', 'belle', 'super', 'beau', 'doctor', 'be..."
25157,5288.0,5288,gyn,m,5,5,5,5,"['delivered', 'baby', 'son', 'ray', '28', 'yea..."
30478,6409.7,6409,gyn,f,5,4,5,5,"['earth', 'good', 'natured', 'point', 'extreme..."
9139,1943.3,1943,-1,m,5,5,5,5,"['family', 'experience', 'wonderful', 'his', '..."


In [3]:
R = [literal_eval(r) for r in list(df['review']) ]
R[0][0]

'fabulous'

In [4]:
df1 = df.groupby(['doc_id','specialty','gender']).apply(lambda x: x.sum())
df1.sample(3)

,,,r_id,doc_id,specialty,gender,S,P,H,K,review
doc_id,specialty,gender,,,,,,,,,
4147,eye,m,37326.6,37323,eyeeyeeyeeyeeyeeyeeyeeyeeye,mmmmmmmmm,45,45,45,45,"['bring', 'great', 'recommend', 'him', 'anyone..."
7470,-1,f,74704.5,74700,-1-1-1-1-1-1-1-1-1-1,ffffffffff,46,50,46,49,"['hodge', 'thorough', 'detected', 'allergy', '..."
6864,-1,f,68644.5,68640,-1-1-1-1-1-1-1-1-1-1,ffffffffff,50,50,50,50,"['2014', '2015', 'received', 'surgery', 'docto..."


In [5]:
counts = dict(df.groupby(['doc_id','specialty','gender']).size())
counts

{(0, 'psych', 'm'): 3,
 (3, 'psych', 'f'): 4,
 (4, '-1', 'f'): 9,
 (6, '-1', 'f'): 10,
 (8, 'family', 'f'): 9,
 (9, 'gyn', 'm'): 10,
 (11, '-1', 'm'): 10,
 (12, 'int', 'f'): 9,
 (14, '-1', 'm'): 4,
 (15, 'skin', 'm'): 10,
 (16, 'skin', 'f'): 10,
 (17, 'family', 'm'): 10,
 (19, 'brain', 'm'): 10,
 (21, 'bone', 'm'): 10,
 (22, 'brain', 'm'): 10,
 (26, '-1', 'm'): 10,
 (29, 'skin', 'f'): 5,
 (30, '-1', 'm'): 10,
 (32, '-1', 'm'): 9,
 (33, '-1', 'f'): 10,
 (34, 'gyn', 'f'): 1,
 (36, 'gyn', 'm'): 10,
 (38, 'bone', 'm'): 10,
 (42, '-1', 'f'): 10,
 (43, '-1', 'm'): 2,
 (44, '-1', 'f'): 10,
 (45, 'int', 'm'): 10,
 (48, '-1', 'f'): 10,
 (56, 'gas', 'f'): 2,
 (57, '-1', 'f'): 10,
 (58, '-1', 'm'): 10,
 (60, 'bone', 'm'): 10,
 (61, 'gas', 'f'): 1,
 (62, 'bone', 'm'): 10,
 (63, 'gyn', 'm'): 10,
 (66, '-1', 'f'): 10,
 (67, '-1', 'm'): 10,
 (72, '-1', 'm'): 10,
 (73, 'pedia', 'f'): 10,
 (74, 'gas', 'f'): 1,
 (75, 'bone', 'm'): 10,
 (76, 'int', 'f'): 2,
 (77, 'skin', 'm'): 10,
 (78, 'eye', 'f'): 10,


In [6]:
d = []
for i,row in df1.iterrows():
    #tup = (row['doc_id'],row['specialty'],row['gender'])
    #print(tup)
    tup = row.name
    count = counts[tup]
    s = round(row['S']/count,1)
    p = round(row['P']/count,1)
    h = round(row['H']/count,1)
    k = round(row['K']/count,1)
    d.append([ str(tup[0]) , tup[1] , tup[2] , [s,p,h,k] ])

In [7]:
d[0]

['0', 'psych', 'm', [5.0, 5.0, 5.0, 5.0]]

In [8]:
len(d),len(df1),len(df)

(4553, 4553, 37646)

In [9]:
reviews = list(df1['review'])

In [10]:
R = [literal_eval(r.replace("][", ",")) for r in reviews]
len(R)

4553

In [11]:
R[0][0]

'fabulous'

In [12]:
G = [i[2] for i in d]
S = [i[1] for i in d]
rating = [i[3] for i in d]

In [13]:
len(R),len(G),len(S),len(rating)

(4553, 4553, 4553, 4553)

In [14]:
scores = [] # scores is -1,0,1
for r in rating:
    score = 0
    for r1 in r:
        if(r1>0 and r1<=5):
            score += r1
    try:
        score = score/4
        #if score > 4.6:
        if score > 4:
            final_score = '1'
        #elif score < 3:
        elif score < 3.5:
            final_score = '-1'
        else:
            final_score = '0'
    except:
        final_score = 'x' # '0'
    scores.append(final_score)
len(scores)

4553

In [15]:
scores_freq = {}
for i in scores:
    try:
        scores_freq[i]+=1
    except:
        scores_freq[i]= 1
scores_freq
# {'1': 22864, '-1': 2583, '0': 553}

{'1': 4076, '-1': 234, '0': 243}

In [16]:
'''R1,S1,G1,rating1 = [],[],[],[]
for i,s in enumerate(S2):
    if s == '-1':
        continue
    R1.append(R2[i])
    S1.append(S2[i])
    G1.append(G2[i])
    rating1.append(scores[i])

R1=R2
S1=S2
G1=G2
rating1=scores'''
R1 = R
S1 = S
G1 = G
rating1 = scores

## Picker

In [17]:
# create reverse dictionary. Scores-> index. Specialties -> index.
revSpcl, revScore = {},{}
revSS = {}
for i in range(len(S1)):
    spcl = S1[i]
    score = rating1[i]
    try:
        revSpcl[spcl].append(i)
    except:
        revSpcl[spcl] = [i]
    try:
        revScore[score].append(i)
    except:
        revScore[score] = [i]
    # JUST TO CHECK IF SOME PAIR IS TOO RARE.
    try:
        revSS[spcl+':'+score+':'+G1[i]]+= 1
    except:
        revSS[spcl+':'+score+':'+G1[i]] = 1
#len(revScore['1'])

In [65]:
# Throw a dice for specialty. Get the corresponding reverse dict.
# Similarly for scores.
# Take intersection. Find a male one. Find a female one.
import random
def picker(roll_rating=False):
    genders = []
    while ('m' not in genders or 'f' not in genders):
        spcl = '-1'
        while(spcl=='-1'): # Ignoring others.
            spcl = random.choice(list(revSpcl.keys()))
        spcl_cand = revSpcl[spcl]
        score = random.choice(list(revScore.keys()))
        score_cand = revScore[score]
        cand = set(spcl_cand).intersection(set(score_cand))
        genders = [G1[i] for i in cand]
    
    #return list(cand)
    #return score_cand
    if roll_rating:
        return list(cand)
    else:
        return list(spcl_cand) # now only one coin toss, that decides specialty. work better? family male -1 has just 11 reviews.

In [66]:
choice = random.choice(picker(roll_rating=True))
print(choice, rating1[choice], S1[choice])

2944 0 psych


## Tag and Online

In [101]:
import gensim
docs = [] # each row is a tagged document.
for i in range(len(R1)):
    docs.append(gensim.models.doc2vec.TaggedDocument(R1[i], [S1[i],G1[i],rating1[i]]))
    #docs.append(gensim.models.doc2vec.TaggedDocument(R1[i], [G1[i],rating1[i]]))
    #docs.append(gensim.models.doc2vec.TaggedDocument(R1[i], [G1[i]]))

In [102]:
#for i in docs:
#    if 'breast' in i[0]:
#        print(i[0])

In [103]:
docs_for_vocab = [] # contains no -1. Not updating docs because then reverese lists must also be updated.
for i in docs:
    if i[1][0] == '-1':
        continue
    else:
        docs_for_vocab.append(i)

In [143]:
repeat = {}
def online(model,batch_size=1,repeat=repeat,roll_rating=False):
    indices = []
    batch = []
    for b in range(batch_size):
        pickm = False
        pickf = False
        cand = picker(roll_rating=roll_rating)
        while(not pickm or not pickf):
            choice = random.choice(cand)
            if G1[choice]=='m':
                if pickm:
                    continue
                else:
                    pickm=True
                    indices.append(choice)
            else:
                if pickf:
                    continue
                else:
                    pickf=True
                    indices.append(choice)
    for i in indices: # only 2: one m, one f.
        try:
            repeat[i]+=1
        except:
            repeat[i]=1
        batch.append(docs[i])
    model.train(batch, total_examples=model.corpus_count)
    

## Hyperparamters

In [107]:
import gensim
import multiprocessing
from gensim.models.doc2vec import Doc2Vec

cores = multiprocessing.cpu_count()
size = 100

In [108]:
def one_model(window,mc,sample,negative,batch_size,num_batches,roll_rating,index,iterations=1,hs=0,dm_concat=1,size=100):
    model = Doc2Vec(dm=0,size=size,window=window,min_count=mc,sample=sample,iter=iterations,workers=cores,hs=hs,negative=negative,dbow_words=1,dm_concat=dm_concat)
    model.build_vocab(docs_for_vocab)
    print(str(model))
    model.intersect_word2vec_format('w2v.bin')
    for i in range(1,num_batches):
        print(i)
        online(model,batch_size=batch_size,roll_rating=roll_rating) # each batch has twice as many as these reviews. m,f.
    version = str(i*batch_size*2)+'_'+str(model)
    version = version[:-1]+',dm_concat'+str(dm_concat)+',rr'+str(roll_rating)+')'
    print(i,end='')
    return model

In [113]:
index = 0
infos = {}
hs = 0
dm_concat = 1
sample = 0
roll_rating = True
window = 20
min_count = 50
negative = 7
num_batches = 20
batch_size = 10000 #10000

m = one_model(window=window,mc=min_count,sample=sample,negative=negative,batch_size=batch_size,num_batches=num_batches,
              roll_rating=roll_rating,index=index,iterations=1,hs=hs,dm_concat=dm_concat,size=100)

Doc2Vec(dbow+w,d100,n7,w20,mc50,t4)
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
19

In [141]:
m.most_similar(positive=[m.docvecs['1'],m.docvecs['m']],topn=15)

[('his', 0.7507585883140564),
 ('he', 0.7432860136032104),
 ('him', 0.7101901173591614),
 ('doctor', 0.6599208116531372),
 ('great', 0.6466248035430908),
 ('professional', 0.6439321637153625),
 ('staff', 0.6182107329368591),
 ('highly', 0.6112654209136963),
 ('thank', 0.6107061505317688),
 ('kind', 0.6000657677650452),
 ('helpful', 0.5997014045715332),
 ('friendly', 0.5994268655776978),
 ('care', 0.5967444181442261),
 ('recommend', 0.5947705507278442),
 ('excellent', 0.5904140472412109)]

In [140]:
m.most_similar(positive=[m.docvecs['1'],m.docvecs['f']],topn=15)

[('her', 0.8369497060775757),
 ('she', 0.81439208984375),
 ('earth', 0.5842311382293701),
 ('doctor', 0.5816746354103088),
 ('professional', 0.5766944885253906),
 ('excellent', 0.5708075761795044),
 ('recommend', 0.5668933987617493),
 ('caring', 0.5610846281051636),
 ('take', 0.5560100078582764),
 ('office', 0.5421832799911499),
 ('friendly', 0.5410971641540527),
 ('love', 0.537408709526062),
 ('always', 0.5352648496627808),
 ('care', 0.5300003290176392),
 ('nice', 0.5293909311294556)]

In [112]:
m.most_similar(positive=[m.docvecs['1']],topn=10)

[('son', 0.6024904847145081),
 ('great', 0.5328873991966248),
 ('well', 0.5252056121826172),
 ('smiling', 0.5094256401062012),
 ('friendly', 0.5070322155952454),
 ('la', 0.5017491579055786),
 ('thorough', 0.49178025126457214),
 ('thank', 0.48956766724586487),
 ('caring', 0.4816704988479614),
 ('en', 0.4807620346546173)]

In [91]:
m.most_similar(positive=[m.docvecs['m']],topn=15)

[('he', 0.8270421028137207),
 ('his', 0.7964839935302734),
 ('him', 0.7369521260261536),
 ('doctor', 0.7111624479293823),
 ('care', 0.6330430507659912),
 ('patient', 0.6259067058563232),
 ('like', 0.5847269296646118),
 ('problem', 0.5820551514625549),
 ('knowledgeable', 0.5800288319587708),
 ('highly', 0.5688095688819885),
 ('good', 0.5687879920005798),
 ('office', 0.5675380229949951),
 ('wealth', 0.5673412084579468),
 ('time', 0.5653829574584961),
 ("he's", 0.5576106309890747)]

In [94]:
len(list(m.docvecs))

2

In [142]:
#docs

In [124]:
#m.docvecs.similarity(m.docvecs['m'], m.docvecs['1'])
import scipy
#m.docvecs['1']

In [137]:
def compare(word):
    print(word,round(1-scipy.spatial.distance.cosine(m.docvecs['m'],m[word]),3),
         round(1-scipy.spatial.distance.cosine(m.docvecs['f'],m[word]),3))

In [139]:
phys = ['beautiful','pretty','expert','brilliant','professional','caring','helpful']

for word in phys:
    try:
        compare(word)
    except:
        print(word)

beautiful 0.221 0.207
pretty 0.241 0.07
expert 0.251 0.212
brilliant 0.271 0.181
professional 0.435 0.337
caring 0.357 0.347
helpful 0.434 0.291
